In [1]:
import os
import shutil

# Path to the ground truth file
gt_file = "ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt"

# Write val_map.txt
with open(gt_file, "r") as f_in, open("val_map.txt", "w") as f_out:
    for i, line in enumerate(f_in):
        label = int(line.strip()) - 1  # Convert 1-based to 0-based
        filename = f"ILSVRC2012_val_{i+1:08d}.JPEG"
        f_out.write(f"{filename}\t{label}\n")

print("✅ val_map.txt generated!")


✅ val_map.txt generated!


In [2]:
import os
import shutil

# === CONFIG ===
val_dir = "val"  # original flat val folder
map_file = "val_map.txt"
output_dir = "val_sorted"

# === STEP 1: Read val_map.txt ===
print("Reading val_map.txt...")
image_to_label = {}
with open(map_file, "r") as f:
    for line in f:
        image_name, label = line.strip().split()
        image_to_label[image_name] = int(label)

# === STEP 2: Create val_sorted/class folders and copy ===
print("Sorting images...")
os.makedirs(output_dir, exist_ok=True)
for image_name, label in image_to_label.items():
    class_dir = os.path.join(output_dir, f"class{label}")
    os.makedirs(class_dir, exist_ok=True)

    src_path = os.path.join(val_dir, image_name)
    dst_path = os.path.join(class_dir, image_name)

    if os.path.exists(src_path):
        shutil.copy2(src_path, dst_path)
    else:
        print(f"WARNING: Image not found: {src_path}")

print("✅ All images sorted into:", output_dir)


Reading val_map.txt...
Sorting images...
✅ All images sorted into: val_sorted


In [3]:
import os, shutil, json, re

# ─── CONFIG ───
val_dir        = "val"                     # your flat folder of 50 000 JPEGs
map_file       = "val_map.txt"             # lines like "ILSVRC2012_val_00000001.JPEG  65"
js_file        = "imagenet_class_index.json" # your JS file
output_dir     = "val_by_synset"           # where to mkdir the 1000 synset folders

# ─── STEP 1: load the JS file as JSON ───
print("Reading imagenet_class_index.js…")
with open(js_file, "r", encoding="utf-8") as f:
    text = f.read()

# Remove any JS-specific wrapping. We want a pure JSON object:
#   e.g. turn 'module.exports = { … };' into '{ … }'
# The simplest regex is to grab the first “{” through the matching final “}”
m = re.search(r"\{[\s\S]*\}", text)
if not m:
    raise RuntimeError("Couldn't find JSON object inside " + js_file)
json_str = m.group(0)
class_index = json.loads(json_str)  # now a dict of strings "0"→[synset, human_name]

# Build an integer→synset dict
idx2syn = {int(k): v[0] for k, v in class_index.items()}

# ─── STEP 2: read val_map.txt → file_name → integer label ───
print("Reading val_map.txt…")
img2lbl = {}
with open(map_file, "r") as f:
    for line in f:
        img_name, lbl_str = line.strip().split()
        img2lbl[img_name] = int(lbl_str)

# ─── STEP 3: copy each JPEG into the right “synset” folder ───
os.makedirs(output_dir, exist_ok=True)
for img_name, lbl in img2lbl.items():
    syn = idx2syn[lbl]                    # e.g. 65 → "n01443537"
    syn_dir = os.path.join(output_dir, syn)
    os.makedirs(syn_dir, exist_ok=True)

    src = os.path.join(val_dir, img_name)
    dst = os.path.join(syn_dir, img_name)
    if os.path.exists(src):
        shutil.copy2(src, dst)
    else:
        print(f"WARNING: {src} not found")

print("✅ Done. All images are in", output_dir)


Reading imagenet_class_index.js…
Reading val_map.txt…
✅ Done. All images are in val_by_synset


In [9]:
import os
import shutil
from scipy.io import loadmat

# ─── CONFIGURE THESE PATHS ───
DEVKIT_PATH    = r"ILSVRC2012_devkit_t12\data"
VAL_IMAGES_DIR = r"val"
OUTPUT_DIR     = r"val_by_synset"

# ─── 1. LOAD META.MAT ─────────────────────────────────────────
print("Step 1: loading meta.mat…")
meta = loadmat(os.path.join(DEVKIT_PATH, "meta.mat"))
synsets_struct = meta["synsets"][0]  # length‐1000 array of structs

# Build (devkit_label [1–1000] → WNID)
devkit_id_to_wnid = {}
for entry in synsets_struct:
    devkit_label = int(entry["ILSVRC2012_ID"][0][0])
    wnid_field   = entry["WNID"][0]

    # If it’s bytes, decode; otherwise convert to str directly
    if isinstance(wnid_field, bytes):
        wnid = wnid_field.decode("utf-8")
    else:
        wnid = str(wnid_field)

    devkit_id_to_wnid[devkit_label] = wnid

assert len(devkit_id_to_wnid) == 1000, "Expect exactly 1000 mappings in meta.mat"

# ─── 2. READ OFFICIAL VALIDATION GROUND TRUTH ────────────────
print("Step 2: reading ILSVRC2012_validation_ground_truth.txt…")
gt_file = os.path.join(DEVKIT_PATH, "ILSVRC2012_validation_ground_truth.txt")
with open(gt_file, "r") as f:
    lines = f.read().strip().splitlines()

if len(lines) != 50000:
    raise RuntimeError(f"Expected 50000 lines, but found {len(lines)}")

# Re‐build the sorted list of 50 000 JPEGs in your “val” folder
all_val_jpegs = sorted(
    [fn for fn in os.listdir(VAL_IMAGES_DIR) if fn.lower().endswith(".jpeg")],
    key=lambda s: s.lower(),
)

if len(all_val_jpegs) != 50000:
    raise RuntimeError(f"Expected 50000 JPEGs, but found {len(all_val_jpegs)}")

# Map each image filename → devkit label (1–1000) by line index
imgname_to_devkit_label = {
    all_val_jpegs[idx]: int(lines[idx])
    for idx in range(len(all_val_jpegs))
}

# ─── 3. COPY IMAGES INTO “val_by_synset/<WNID>” ──────────────
print("Step 3: copying images into synset‐named folders…")
os.makedirs(OUTPUT_DIR, exist_ok=True)

for img_filename, devkit_label in imgname_to_devkit_label.items():
    wnid = devkit_id_to_wnid[devkit_label]  # e.g. "n01443537"
    synset_folder = os.path.join(OUTPUT_DIR, wnid)
    os.makedirs(synset_folder, exist_ok=True)

    src_path = os.path.join(VAL_IMAGES_DIR, img_filename)
    dst_path = os.path.join(synset_folder, img_filename)

    if not os.path.exists(src_path):
        print(f"  WARNING: {src_path} does not exist—skipping.")
        continue

    shutil.copy2(src_path, dst_path)

print("✅ Done. All validation images are now under 1000 synset folders in", OUTPUT_DIR)


Step 1: loading meta.mat…


AssertionError: Expect exactly 1000 mappings in meta.mat

In [5]:
import os

print("PWD =", os.getcwd())
print("Contents of current dir: ", os.listdir())
print("Expected meta.mat path exists? ", os.path.exists(os.path.join("ILSVRC2012_devkit_t12", "data", "meta.mat")))


PWD = d:\UniGe\2\DL\Project
Contents of current dir:  ['ILSVRC2012_devkit_t12', 'ILSVRC2012_devkit_t12.tar.gz', 'ILSVRC2012_img_test_v10102019.tar', 'ILSVRC2012_img_val.tar', 'ILSVRC2012_validation_ground_truth.txt', 'imagenet_class_index.json', 'Implementation.docx', 'map_clsloc.txt', 'ProjectPresentation.pdf', 'sorting_val.ipynb', 'Swin Fine-tuned Models', 'Swin Transformer Hierarchical Vision Transformer using ShiftedWindows.pdf', 'Swin-Transformer', 'val', 'val_map.txt', 'val_sorted', '~$plementation.docx', '~WRL1713.tmp']
Expected meta.mat path exists?  True


In [1]:
import torch
torch.version.__version__  # Ensure PyTorch is imported correctly
# import timm
# timm.__version__

'2.4.0+cu124'

In [1]:
import torchvision
torchvision.__version__  # Ensure torchvision is imported correctly

'0.19.0+cu124'

In [1]:
import torchvision

print(torchvision.__version__, torchvision.cuda.is_available())

AttributeError: module 'torchvision' has no attribute 'cuda'